In [1]:
# Import functions needed for this notebook
import pandas as pd
import statsmodels.api as sm
from statsmodels.genmod.families import NegativeBinomial
from statsmodels.discrete.discrete_model import NegativeBinomial as CountNB
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
# Load the dataset
df = pd.read_excel("thesis_dataset.xlsx")
df.head()

,player_name,highest_ranking,country,mother_occupation,father_occupation,sibling_info,sibling_player,titles_won,association,year_turned_pro,father_occupation_std,mother_occupation_std,father_ISEI,mother_ISEI,family_ISEI
0,Roger Federer,1,Switzerland,employee at pharmaceutical firm,executive at pharmaceutical firm,True,False,103,ATP,1998,Managing directors and chief executives,Pharmaceutical technicians and assistants,70,40,55
1,Taylor Fritz,4,USA,professional tennis player,professional tennis player,True,False,8,ATP,2015,Athletes and sports players,Athletes and sports players,46,46,46
2,Novak Djokovic,1,Serbia,entrepreneur,professional skier,True,True,99,ATP,2003,Athletes and sports players,Managing directors and chief executives,46,70,58
3,Jessica Pegula,3,USA,CEO,business owner,True,False,8,WTA,2009,Managing directors and chief executives,Managing directors and chief executives,70,70,70
4,Grigor Dimitrov,3,Bulgaria,volleyball player,tennis coach,False,False,9,ATP,2008,Athletes and sports players,Athletes and sports players,46,46,46


In [3]:
# Convert boolean columns to integers
df['sibling_info'] = df['sibling_info'].astype(int)
df['sibling_player'] = df['sibling_player'].astype(int)

df.head()

,player_name,highest_ranking,country,mother_occupation,father_occupation,sibling_info,sibling_player,titles_won,association,year_turned_pro,father_occupation_std,mother_occupation_std,father_ISEI,mother_ISEI,family_ISEI
0,Roger Federer,1,Switzerland,employee at pharmaceutical firm,executive at pharmaceutical firm,1,0,103,ATP,1998,Managing directors and chief executives,Pharmaceutical technicians and assistants,70,40,55
1,Taylor Fritz,4,USA,professional tennis player,professional tennis player,1,0,8,ATP,2015,Athletes and sports players,Athletes and sports players,46,46,46
2,Novak Djokovic,1,Serbia,entrepreneur,professional skier,1,1,99,ATP,2003,Athletes and sports players,Managing directors and chief executives,46,70,58
3,Jessica Pegula,3,USA,CEO,business owner,1,0,8,WTA,2009,Managing directors and chief executives,Managing directors and chief executives,70,70,70
4,Grigor Dimitrov,3,Bulgaria,volleyball player,tennis coach,0,0,9,ATP,2008,Athletes and sports players,Athletes and sports players,46,46,46


In [4]:
# Create the association dummy (ATP baseline, keep WTA)
df['association_WTA'] = (df['association'] == 'WTA').astype(int)

In [5]:
# Get the top 5 countries by the number of players and use them as dummy variables
top_countries = df['country'].value_counts().head(5).index.tolist()
for c in top_countries:
    df[f'country_{c.replace(" ","_")}'] = (df['country']==c).astype(int)

In [6]:
df.head()

,player_name,highest_ranking,country,mother_occupation,father_occupation,sibling_info,sibling_player,titles_won,association,year_turned_pro,...,mother_occupation_std,father_ISEI,mother_ISEI,family_ISEI,association_WTA,country_USA,country_Russia,country_Australia,country_France,country_Spain
0,Roger Federer,1,Switzerland,employee at pharmaceutical firm,executive at pharmaceutical firm,1,0,103,ATP,1998,...,Pharmaceutical technicians and assistants,70,40,55,0,0,0,0,0,0
1,Taylor Fritz,4,USA,professional tennis player,professional tennis player,1,0,8,ATP,2015,...,Athletes and sports players,46,46,46,0,1,0,0,0,0
2,Novak Djokovic,1,Serbia,entrepreneur,professional skier,1,1,99,ATP,2003,...,Managing directors and chief executives,46,70,58,0,0,0,0,0,0
3,Jessica Pegula,3,USA,CEO,business owner,1,0,8,WTA,2009,...,Managing directors and chief executives,70,70,70,1,1,0,0,0,0
4,Grigor Dimitrov,3,Bulgaria,volleyball player,tennis coach,0,0,9,ATP,2008,...,Athletes and sports players,46,46,46,0,0,0,0,0,0


#### Models having `titles_won` as dependent variable

In [7]:
# Create and fit a regression model where the dependent variable is 'titles_won' and the independent variable is 'mother_ISEI'
model_mother_ISEI = sm.GLM.from_formula("titles_won ~ mother_ISEI", data=df, family=NegativeBinomial()).fit()

# Print the summary of the model
print(model_mother_ISEI.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             titles_won   No. Observations:                  100
Model:                            GLM   Df Residuals:                       98
Model Family:        NegativeBinomial   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -434.26
Date:                Sun, 08 Jun 2025   Deviance:                       123.78
Time:                        13:45:12   Pearson chi2:                     139.
No. Iterations:                     8   Pseudo R-squ. (CS):            0.05785
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       4.1620      0.316     13.154      

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [8]:
# Create and fit a regression model where the dependent variable is `titles_won` and the independent variable is `father_ISEI`
model_father_ISEI = sm.GLM.from_formula("titles_won ~ father_ISEI", data=df, family=NegativeBinomial()).fit()

# Print the summary of the model
print(model_father_ISEI.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             titles_won   No. Observations:                  100
Model:                            GLM   Df Residuals:                       98
Model Family:        NegativeBinomial   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -437.22
Date:                Sun, 08 Jun 2025   Deviance:                       129.71
Time:                        13:45:12   Pearson chi2:                     155.
No. Iterations:                     6   Pseudo R-squ. (CS):          0.0003240
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       3.2902      0.390      8.431      

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [9]:
# Sequentially add predictors to the model, one at a time
model_step1 = sm.GLM.from_formula("titles_won ~ mother_ISEI", data=df, family=NegativeBinomial()).fit()

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [10]:
model_step2 = sm.GLM.from_formula("titles_won ~ mother_ISEI + father_ISEI", data=df, family=NegativeBinomial()).fit()

# Print the summary of the model
print(model_step2.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             titles_won   No. Observations:                  100
Model:                            GLM   Df Residuals:                       97
Model Family:        NegativeBinomial   Df Model:                            2
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -434.02
Date:                Sun, 08 Jun 2025   Deviance:                       123.30
Time:                        13:45:12   Pearson chi2:                     143.
No. Iterations:                     8   Pseudo R-squ. (CS):            0.06241
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       3.9701      0.459      8.658      

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [11]:
# Calculate the VIF for model_step2 (without constant)
print("VIF for model_step2:")
vif_step2 = pd.DataFrame({
    'Variable': model_step2.model.exog_names[1:],  # Exclude the constant
    'VIF': [variance_inflation_factor(model_step2.model.exog, i) for i in range(1, model_step2.model.exog.shape[1])]
})
print(vif_step2)

VIF for model_step2:
      Variable       VIF
0  mother_ISEI  1.030596
1  father_ISEI  1.030596


In [12]:
model_3 = sm.GLM.from_formula("titles_won ~ mother_ISEI + father_ISEI + sibling_info", data=df, family=NegativeBinomial()).fit()

# Print the summary of the model
print(model_3.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             titles_won   No. Observations:                  100
Model:                            GLM   Df Residuals:                       96
Model Family:        NegativeBinomial   Df Model:                            3
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -432.39
Date:                Sun, 08 Jun 2025   Deviance:                       120.04
Time:                        13:45:12   Pearson chi2:                     140.
No. Iterations:                     8   Pseudo R-squ. (CS):            0.09244
Covariance Type:            nonrobust                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        4.4021      0.531      8.295   

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [13]:
# Calculate the VIF for model_3 (without constant)
print("VIF for model_3:")
vif_model_3 = pd.DataFrame({
    'Variable': model_3.model.exog_names[1:],  # Exclude the constant
    'VIF': [variance_inflation_factor(model_3.model.exog, i) for i in range(1, model_3.model.exog.shape[1])]
})
print(vif_model_3)

VIF for model_3:
       Variable       VIF
0   mother_ISEI  1.079099
1   father_ISEI  1.034383
2  sibling_info  1.047620


In [14]:
model_step4 = sm.GLM.from_formula("titles_won ~ mother_ISEI + father_ISEI + sibling_info + sibling_player", data=df, family=NegativeBinomial()).fit()

# Print the summary of the model
print(model_step4.summary())

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                 Generalized Linear Model Regression Results                  
Dep. Variable:             titles_won   No. Observations:                  100
Model:                            GLM   Df Residuals:                       95
Model Family:        NegativeBinomial   Df Model:                            4
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -432.32
Date:                Sun, 08 Jun 2025   Deviance:                       119.90
Time:                        13:45:12   Pearson chi2:                     140.
No. Iterations:                     8   Pseudo R-squ. (CS):            0.09375
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          4.3982      0.531      8.

In [15]:
# Calculate the VIF for model_step4 (without constant)
print("VIF for model_step4:")
vif_model_step4 = pd.DataFrame({
    'Variable': model_step4.model.exog_names[1:],  # Exclude the constant
    'VIF': [variance_inflation_factor(model_step4.model.exog, i) for i in range(1, model_step4.model.exog.shape[1])]
})
print(vif_model_step4)

VIF for model_step4:
         Variable       VIF
0     mother_ISEI  1.080426
1     father_ISEI  1.034398
2    sibling_info  1.183954
3  sibling_player  1.132279


In [16]:
model_step5 = sm.GLM.from_formula("titles_won ~ mother_ISEI + father_ISEI + sibling_info + sibling_player + association_WTA", data=df, family=NegativeBinomial()).fit()

# Print the summary of the model
print(model_step5.summary())

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                 Generalized Linear Model Regression Results                  
Dep. Variable:             titles_won   No. Observations:                  100
Model:                            GLM   Df Residuals:                       94
Model Family:        NegativeBinomial   Df Model:                            5
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -431.70
Date:                Sun, 08 Jun 2025   Deviance:                       118.66
Time:                        13:45:12   Pearson chi2:                     140.
No. Iterations:                    11   Pseudo R-squ. (CS):             0.1049
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           4.1838      0.541     

In [17]:
country_dummies = " + ".join([f"country_{c.replace(' ', '_')}" for c in top_countries])
model_step6 = sm.GLM.from_formula(f"titles_won ~ mother_ISEI + father_ISEI + sibling_info + sibling_player + association_WTA + {country_dummies}", data=df, family=NegativeBinomial()).fit()

# Print the summary of the model
print(model_step6.summary())

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                 Generalized Linear Model Regression Results                  
Dep. Variable:             titles_won   No. Observations:                  100
Model:                            GLM   Df Residuals:                       89
Model Family:        NegativeBinomial   Df Model:                           10
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -427.96
Date:                Sun, 08 Jun 2025   Deviance:                       111.19
Time:                        13:45:12   Pearson chi2:                     117.
No. Iterations:                    13   Pseudo R-squ. (CS):             0.1694
Covariance Type:            nonrobust                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             4.1680      0.54

In [18]:
# Create the full model with all predictors excluding `sibling_player` from the independent variables
full_model = sm.GLM.from_formula(f"titles_won ~ mother_ISEI + father_ISEI + sibling_info + association_WTA + {country_dummies}", data=df, family=NegativeBinomial()).fit()

# Print the summary of the full model
print("\nFull model excluding sibling_player:\n")
print(full_model.summary())

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "



Full model excluding sibling_player:

                 Generalized Linear Model Regression Results                  
Dep. Variable:             titles_won   No. Observations:                  100
Model:                            GLM   Df Residuals:                       90
Model Family:        NegativeBinomial   Df Model:                            9
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -427.96
Date:                Sun, 08 Jun 2025   Deviance:                       111.19
Time:                        13:45:12   Pearson chi2:                     117.
No. Iterations:                    12   Pseudo R-squ. (CS):             0.1693
Covariance Type:            nonrobust                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------

In [19]:
# Calculate the VIF for full_model (without constant)
print("VIF for full_model:")
vif_full_model = pd.DataFrame({
    'Variable': full_model.model.exog_names[1:],  # Exclude the constant
    'VIF': [variance_inflation_factor(full_model.model.exog, i) for i in range(1, full_model.model.exog.shape[1])]
})
print(vif_full_model)


VIF for full_model:
            Variable       VIF
0        mother_ISEI  1.140882
1        father_ISEI  1.072908
2       sibling_info  1.102780
3    association_WTA  1.031985
4        country_USA  1.154539
5     country_Russia  1.099623
6  country_Australia  1.071131
7     country_France  1.091634
8      country_Spain  1.073521


In [20]:
# Define a list of formulas, building up predictors step by step
formulas = [
    "titles_won ~ mother_ISEI",
    "titles_won ~ mother_ISEI + father_ISEI",
    "titles_won ~ mother_ISEI + father_ISEI + sibling_info",
    "titles_won ~ mother_ISEI + father_ISEI + sibling_info + association_WTA",
    f"titles_won ~ mother_ISEI + father_ISEI + sibling_info + association_WTA + {country_dummies}"
]

In [21]:
# Create a dictionary to collect results (model name, formula, AIC, BIC, the coefficients and standard errors for each variable)
results_summary = {
    'Model': [],
    'Formula': [],
    'AIC': [],
    'BIC': [],
    'Coef_mother_ISEI': [],
    'SE_mother_ISEI': [],
    'Coef_father_ISEI': [],
    'SE_father_ISEI': [],
    'Coef_sibling_info': [],
    'SE_sibling_info': [],
    'Coef_sibling_player': [],
    'SE_sibling_player': []
}

In [22]:
# Fit each model and collect results
for idx, formula in enumerate(formulas, start=1):
    model = sm.GLM.from_formula(formula, data=df, family=NegativeBinomial()).fit()
    
    results_summary['Model'].append(f"Model {idx}")
    results_summary['Formula'].append(formula)
    results_summary['AIC'].append(model.aic)
    results_summary['BIC'].append(model.bic)
    
    # For each key variable, record coef and SE if present; otherwise NaN
    for var in ['mother_ISEI', 'father_ISEI', 'sibling_info', 'sibling_player']:
        if var in model.params.index:
            results_summary[f'Coef_{var}'].append(model.params[var])
            results_summary[f'SE_{var}'].append(model.bse[var])
        else:
            results_summary[f'Coef_{var}'].append(float('nan'))
            results_summary[f'SE_{var}'].append(float('nan'))

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1838: FutureWarning: The bic value is computed using the deviance formula. After 0.13 this will change to the log-likelihood based formula. This change has no impact on the relative rank of models compared using BIC. You can directly access the log-likelihood version using the `bic_llf` attribute. You can suppress this message by calling statsmodels.genmod.generalized_linear_model.SET_USE_BIC_LLF with True to get the LLF-based version now or False to retainthe deviance version.
  warnings.warn(
c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. U

In [23]:
# Convert the results summary to a DataFrame
results_df = pd.DataFrame(results_summary)

# Display the results DataFrame
print("\nResults Summary DataFrame:")
print(results_df)


Results Summary DataFrame:
     Model                                            Formula         AIC  \
0  Model 1                           titles_won ~ mother_ISEI  872.522023   
1  Model 2             titles_won ~ mother_ISEI + father_ISEI  874.036576   
2  Model 3  titles_won ~ mother_ISEI + father_ISEI + sibli...  872.781469   
3  Model 4  titles_won ~ mother_ISEI + father_ISEI + sibli...  873.810899   
4  Model 5  titles_won ~ mother_ISEI + father_ISEI + sibli...  875.929589   

          BIC  Coef_mother_ISEI  SE_mother_ISEI  Coef_father_ISEI  \
0 -327.522092         -0.017323        0.006215               NaN   
1 -323.402368         -0.018580        0.006304          0.004720   
2 -322.052305         -0.016021        0.006453          0.003697   
3 -318.417705         -0.015137        0.006455          0.004541   
4 -303.273164         -0.013871        0.006667          0.004335   

   SE_father_ISEI  Coef_sibling_info  SE_sibling_info  Coef_sibling_player  \
0             Na

#### Models having `highest_ranking` as the dependent variable

In [24]:
# Create and fit a regression model where the dependent variable is 'highest_ranking' and the independent variable is 'mother_ISEI'
model2_mother_ISEI = sm.GLM.from_formula("highest_ranking ~ mother_ISEI", data=df, family=NegativeBinomial()).fit()

# Print the summary of the model
print(model2_mother_ISEI.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:        highest_ranking   No. Observations:                  100
Model:                            GLM   Df Residuals:                       98
Model Family:        NegativeBinomial   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -273.42
Date:                Sat, 07 Jun 2025   Deviance:                       101.44
Time:                        19:32:23   Pearson chi2:                     182.
No. Iterations:                    11   Pseudo R-squ. (CS):            0.08539
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.7636      0.354      2.155      

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [25]:
# Create and fit a regression model where the dependent variable is `highest_ranking` and the independent variable is `father_ISEI`
model2_father_ISEI = sm.GLM.from_formula("highest_ranking ~ father_ISEI", data=df, family=NegativeBinomial()).fit()

# Print the summary of the model
print(model2_father_ISEI.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:        highest_ranking   No. Observations:                  100
Model:                            GLM   Df Residuals:                       98
Model Family:        NegativeBinomial   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -275.27
Date:                Sat, 07 Jun 2025   Deviance:                       105.14
Time:                        19:32:54   Pearson chi2:                     219.
No. Iterations:                     8   Pseudo R-squ. (CS):            0.05091
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       2.6585      0.416      6.392      

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [26]:
# Sequentially add predictors to the model, one at a time
model2_step1 = sm.GLM.from_formula("highest_ranking ~ mother_ISEI", data=df, family=NegativeBinomial()).fit()

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [27]:
model2_step2 = sm.GLM.from_formula("highest_ranking ~ mother_ISEI + father_ISEI", data=df, family=NegativeBinomial()).fit()

# Print the summary of the model
print(model2_step2.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:        highest_ranking   No. Observations:                  100
Model:                            GLM   Df Residuals:                       97
Model Family:        NegativeBinomial   Df Model:                            2
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -270.42
Date:                Sat, 07 Jun 2025   Deviance:                       95.426
Time:                        19:34:07   Pearson chi2:                     157.
No. Iterations:                    15   Pseudo R-squ. (CS):             0.1387
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       1.7997      0.496      3.630      

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [28]:
# Calculate the VIF for model_step2 (without constant)
print("VIF for model_step2:")
vif_step2 = pd.DataFrame({
    'Variable': model_step2.model.exog_names[1:],  # Exclude the constant
    'VIF': [variance_inflation_factor(model2_step2.model.exog, i) for i in range(1, model2_step2.model.exog.shape[1])]
})
print(vif_step2)

VIF for model_step2:
      Variable       VIF
0  mother_ISEI  1.030596
1  father_ISEI  1.030596


In [29]:
model2_step3 = sm.GLM.from_formula("highest_ranking ~ mother_ISEI + father_ISEI + sibling_info", data=df, family=NegativeBinomial()).fit()

# Print the summary of the model
print(model2_step3.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:        highest_ranking   No. Observations:                  100
Model:                            GLM   Df Residuals:                       96
Model Family:        NegativeBinomial   Df Model:                            3
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -268.93
Date:                Sat, 07 Jun 2025   Deviance:                       92.459
Time:                        19:34:54   Pearson chi2:                     148.
No. Iterations:                    15   Pseudo R-squ. (CS):             0.1639
Covariance Type:            nonrobust                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.1280      0.597      1.890   

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [30]:
# Calculate the VIF for model_3 (without constant)
print("VIF for model_3:")
vif_model_3 = pd.DataFrame({
    'Variable': model2_step3.model.exog_names[1:],  # Exclude the constant
    'VIF': [variance_inflation_factor(model2_step3.model.exog, i) for i in range(1, model2_step3.model.exog.shape[1])]
})
print(vif_model_3)

VIF for model_3:
       Variable       VIF
0   mother_ISEI  1.079099
1   father_ISEI  1.034383
2  sibling_info  1.047620


In [31]:
model2_step4 = sm.GLM.from_formula("highest_ranking ~ mother_ISEI + father_ISEI + sibling_info + sibling_player", data=df, family=NegativeBinomial()).fit()

# Print the summary of the model
print(model2_step4.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:        highest_ranking   No. Observations:                  100
Model:                            GLM   Df Residuals:                       95
Model Family:        NegativeBinomial   Df Model:                            4
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -268.91
Date:                Sat, 07 Jun 2025   Deviance:                       92.422
Time:                        19:35:32   Pearson chi2:                     149.
No. Iterations:                    21   Pseudo R-squ. (CS):             0.1642
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          1.1236      0.597      1.

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [32]:
# Calculate the VIF for model_step4 (without constant)
print("VIF for model_step4:")
vif_model_step4 = pd.DataFrame({
    'Variable': model_step4.model.exog_names[1:],  # Exclude the constant
    'VIF': [variance_inflation_factor(model2_step4.model.exog, i) for i in range(1, model2_step4.model.exog.shape[1])]
})
print(vif_model_step4)

VIF for model_step4:
         Variable       VIF
0     mother_ISEI  1.080426
1     father_ISEI  1.034398
2    sibling_info  1.183954
3  sibling_player  1.132279


In [33]:
model2_step5 = sm.GLM.from_formula("highest_ranking ~ mother_ISEI + father_ISEI + sibling_info + sibling_player + association_WTA", data=df, family=NegativeBinomial()).fit()

# Print the summary of the model
print(model2_step5.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:        highest_ranking   No. Observations:                  100
Model:                            GLM   Df Residuals:                       94
Model Family:        NegativeBinomial   Df Model:                            5
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -264.18
Date:                Sat, 07 Jun 2025   Deviance:                       82.953
Time:                        19:36:14   Pearson chi2:                     126.
No. Iterations:                    14   Pseudo R-squ. (CS):             0.2397
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           1.4585      0.604     

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [34]:
country_dummies = " + ".join([f"country_{c.replace(' ', '_')}" for c in top_countries])
model2_step6 = sm.GLM.from_formula(f"highest_ranking ~ mother_ISEI + father_ISEI + sibling_info + sibling_player + association_WTA + {country_dummies}", data=df, family=NegativeBinomial()).fit()

# Print the summary of the model
print(model2_step6.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:        highest_ranking   No. Observations:                  100
Model:                            GLM   Df Residuals:                       89
Model Family:        NegativeBinomial   Df Model:                           10
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -260.64
Date:                Sat, 07 Jun 2025   Deviance:                       75.871
Time:                        19:36:40   Pearson chi2:                     109.
No. Iterations:                    14   Pseudo R-squ. (CS):             0.2917
Covariance Type:            nonrobust                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             1.2448      0.61

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [35]:
# Create the full model with all predictors excluding `sibling_player` from the independent variables
full_model2 = sm.GLM.from_formula(f"highest_ranking ~ mother_ISEI + father_ISEI + sibling_info + association_WTA + {country_dummies}", data=df, family=NegativeBinomial()).fit()

# Print the summary of the full model
print("\nFull model excluding sibling_player:\n")
print(full_model2.summary())


Full model excluding sibling_player:

                 Generalized Linear Model Regression Results                  
Dep. Variable:        highest_ranking   No. Observations:                  100
Model:                            GLM   Df Residuals:                       90
Model Family:        NegativeBinomial   Df Model:                            9
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -260.68
Date:                Sat, 07 Jun 2025   Deviance:                       75.958
Time:                        19:37:10   Pearson chi2:                     110.
No. Iterations:                    13   Pseudo R-squ. (CS):             0.2911
Covariance Type:            nonrobust                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [36]:
# Calculate the VIF for full_model (without constant)
print("VIF for full_model:")
vif_full_model = pd.DataFrame({
    'Variable': full_model2.model.exog_names[1:],  # Exclude the constant
    'VIF': [variance_inflation_factor(full_model2.model.exog, i) for i in range(1, full_model2.model.exog.shape[1])]
})
print(vif_full_model)

VIF for full_model:
            Variable       VIF
0        mother_ISEI  1.140882
1        father_ISEI  1.072908
2       sibling_info  1.102780
3    association_WTA  1.031985
4        country_USA  1.154539
5     country_Russia  1.099623
6  country_Australia  1.071131
7     country_France  1.091634
8      country_Spain  1.073521


In [38]:
# Define a list of formulas, building up predictors step by step
formulas2 = [
    "highest_ranking ~ mother_ISEI",
    "highest_ranking ~ father_ISEI",
    "highest_ranking ~ mother_ISEI + father_ISEI",
    "highest_ranking ~ mother_ISEI + father_ISEI + sibling_info",
    "highest_ranking ~ mother_ISEI + father_ISEI + sibling_info + association_WTA",
    f"highest_ranking ~ mother_ISEI + father_ISEI + sibling_info + association_WTA + {country_dummies}"
]

In [39]:
# Create a dictionary to collect results (model name, formula, AIC, BIC, the coefficients and standard errors for each variable)
results_summary2 = {
    'Model': [],
    'Formula': [],
    'AIC': [],
    'BIC': [],
    'Coef_mother_ISEI': [],
    'SE_mother_ISEI': [],
    'Coef_father_ISEI': [],
    'SE_father_ISEI': [],
    'Coef_sibling_info': [],
    'SE_sibling_info': [],
    'Coef_sibling_player': [],
    'SE_sibling_player': []
}

In [40]:
# Fit each model and collect results
for idx, formula in enumerate(formulas2, start=1):
    model = sm.GLM.from_formula(formula, data=df, family=NegativeBinomial()).fit()

    results_summary2['Model'].append(f"Model {idx}")
    results_summary2['Formula'].append(formula)
    results_summary2['AIC'].append(model.aic)
    results_summary2['BIC'].append(model.bic)

    # For each key variable, record coef and SE if present; otherwise NaN
    for var in ['mother_ISEI', 'father_ISEI', 'sibling_info', 'sibling_player']:
        if var in model.params.index:
            results_summary2[f'Coef_{var}'].append(model.params[var])
            results_summary2[f'SE_{var}'].append(model.bse[var])
        else:
            results_summary2[f'Coef_{var}'].append(float('nan'))
            results_summary2[f'SE_{var}'].append(float('nan'))

c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1838: FutureWarning: The bic value is computed using the deviance formula. After 0.13 this will change to the log-likelihood based formula. This change has no impact on the relative rank of models compared using BIC. You can directly access the log-likelihood version using the `bic_llf` attribute. You can suppress this message by calling statsmodels.genmod.generalized_linear_model.SET_USE_BIC_LLF with True to get the LLF-based version now or False to retainthe deviance version.
  warnings.warn(
c:\Users\peta_\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. U

In [41]:
# Convert the results summary to a DataFrame
results2_df = pd.DataFrame(results_summary2)

# Display the results DataFrame
print("\nResults Summary DataFrame:")
print(results2_df)


Results Summary DataFrame:
     Model                                            Formula         AIC  \
0  Model 1                      highest_ranking ~ mother_ISEI  550.841717   
1  Model 2                      highest_ranking ~ father_ISEI  554.542424   
2  Model 3        highest_ranking ~ mother_ISEI + father_ISEI  546.832220   
3  Model 4  highest_ranking ~ mother_ISEI + father_ISEI + ...  545.865198   
4  Model 5  highest_ranking ~ mother_ISEI + father_ISEI + ...  538.525778   
5  Model 6  highest_ranking ~ mother_ISEI + father_ISEI + ...  541.364098   

          BIC  Coef_mother_ISEI  SE_mother_ISEI  Coef_father_ISEI  \
0 -349.870974          0.018076        0.006851               NaN   
1 -346.170267               NaN             NaN         -0.018820   
2 -351.275301          0.018769        0.007103         -0.020864   
3 -349.637152          0.017500        0.007268         -0.019803   
4 -354.371402          0.016787        0.007293         -0.016836   
5 -338.507231     